# CIS 545 Homework 4 Advanced

## Step 1: Local Sequence Alignment

In information theory, linguistics and computer science, the Levenshtein distance is a string metric for measuring the difference between two sequences and has many applications in other disciplines also. There are two popular sequence alignment algorithms - **Needleman–Wunsch algorithm** (global alignment) and **Smith–Waterman algorithm** (local alignment). 

In this homework, we shall expand our knowledge of Levenshtein Distance with Dynamic Programming by examining another variant of the Levenshtein algorithm called the **Smith–Waterman algorithm**. The Smith–Waterman algorithm performs local sequence alignment; that is, for determining similar regions between two strings of nucleic acid sequences or protein sequences. Instead of looking at the entire sequence, the Smith–Waterman algorithm compares segments of all possible lengths and optimizes the similarity measure. From [Wikipedia](https://en.wikipedia.org/wiki/Sequence_alignment), a sequence alignment is a way of arranging genetic sequences to identify regions of similarity.

To “align” two sequences, we insert gaps in them until their similar substrings line up. Adding these gaps helps to align sequences that may have been changed through mutations such as insertions and deletions, AKA **indels**. Because there could be potentially many ways to align two sequences, the Smith–Waterman algorithm allows for “scoring” of sequence alignments. The scoring system allows one to determine an ordering on alignments, i.e. determine which alignments are better than others. 

The main difference to the Needleman–Wunsch algorithm is that negative scoring matrix cells are set to zero, which renders the (thus positively scoring) local alignments visible. **Traceback procedure starts at the highest scoring matrix cell and proceeds until a cell with score zero is encountered, yielding the highest scoring local alignment.**

In [1]:
# Import your packages here - make sure to have pandas and numpy, and anything else?
import pandas as pd
import numpy as np

# If you use a non-standard package, be sure to install it here too

In [2]:
# Your reference sequences

seq1 = ['G', 'G', 'T', 'T', 'G', 'A', 'C', 'T', 'A']
seq2 = ['T', 'G', 'T', 'T', 'A', 'C', 'G', 'G']

In [3]:
# These functions are used for grading

def score_grid(D, seq1, seq2):
    D_arr = np.zeros((len(seq1)+1,len(seq2)+1), dtype=int)
    for i in range(len(seq1)+1): 
        for j in range(len(seq2)+1): 
            D_arr[i][j] = D[(j,i)]
    return D_arr

def directions_grid(D, seq1, seq2):
    D_arr = dict()
    for j in range(len(seq2)+1): 
        D_arr[j] = dict()
        for i in range(len(seq1)+1): 
            D_arr[j][i] = D[(i,j)][1]
    return pd.DataFrame(D_arr)

## Step 1.1: Implementing Smith–Waterman

Here, we provide a rough sketch of the pseudocode of Smith–Waterman below (based off of [Wikipedia](https://en.wikipedia.org/wiki/Smith%E2%80%93Waterman_algorithm#Implementation)):

Let $D$ refer to the table of alignment scores computed by the algorithm (note that if $m = \text{len(seq1)}$ and $n = \text{len(seq2)}$, then $D$ has dimensions $(m+1) \times (n+1)$). 

**Initialization**  
`Let D(i, 0) = 0 for i <- 0..len(seq1) + 1`  
`Let D(0, j) = 0 for j <- 0..len(seq2) + 1`

**Update Step**  
`D(i, j) = max{D(i - 1, j    ) - indelCost,  
               D(i    , j - 1) - indelCost,
               D(i - 1, j - 1) + match_mismatch_cost(seq1_{i-1}, seq2_{j-1}),
               0}`
               
Note that when you implement this, it will be easiest to represent $D$ as a dictionary instead of a matrix, for reasons you will see later.

For this homework, assume these values:
Indel Cost (Gap Penalty) = 2, 
Match Cost = 3,
Mismatch Cost = -3.

Please refer to Wikipedia page to know more about these terms.

Write a function called `match_mismatch_cost(n1, n2)` that returns match_cost (3 in this case) if the characters provided as arguments match, otherwise it returns mismatch_cost (-3 in this case).

In [4]:
# TODO: Write the function match_mismatch_cost(n1, n2) here.

def match_mismatch_cost(n1, n2):
    if n1==n2:
        return 3
    if n1!=n2:
        return -3

In [5]:
# CIS 545 Test Case

assert(match_mismatch_cost('a', 'a') == 3)


In [6]:
print('CIS 545 Test Case Padding')

CIS 545 Test Case Padding


Now, write the function `calculate_scores(seq2, seq1, indelCost)` that returns a dictionary, `D`, that corresponds to the table of alignment scores computed by the Smith–Waterman algorithm. Note the order of the entries.

The dictionary should have int tuples `(i, j)` as keys and, as the corresponding mappings, ints that correspond to the alignment score of `seq1[0:i]` with `seq2[0:j]`. 

In [7]:
# TODO: Write the function calculate_scores(seq2, seq1, indelCost) here.

def calculate_scores(seq2, seq1, indelCost):
    D=dict()
    len1 = len(seq1)
    len2 = len(seq2)
#     print(len1,len2)
    for i in range(len1+1):
        D[(i,0)]=0
    for j in range(len2+1):
        D[(0,j)]=0
    for i in range(1,len1+1):
        for j in range(1,len2+1):
#             print(i,j)
#             D[(i,j)]=0
            D[(i,j)] = max((D[(i-1,j)]-indelCost),(D[(i,j-1)]-indelCost),(D[(i-1,j-1)]+match_mismatch_cost(seq1[i-1],seq2[j-1])),0)
#             print(D[(i,j)])
    return D                      

In [8]:
# CIS 545 Test Case
# Yes, we are supposed to run the seq in reverse order!

indelCost = 2
D = calculate_scores(seq1, seq2, indelCost)
D_arr = score_grid(D, seq1, seq2)
display(D_arr)

assert((D_arr[8] == np.array([0, 3, 1, 5, 4, 6, 11, 10, 8])).all())

array([[ 0,  0,  0,  0,  0,  0,  0,  0,  0],
       [ 0,  0,  3,  1,  0,  0,  0,  3,  3],
       [ 0,  0,  3,  1,  0,  0,  0,  3,  6],
       [ 0,  3,  1,  6,  4,  2,  0,  1,  4],
       [ 0,  3,  1,  4,  9,  7,  5,  3,  2],
       [ 0,  1,  6,  4,  7,  6,  4,  8,  6],
       [ 0,  0,  4,  3,  5, 10,  8,  6,  5],
       [ 0,  0,  2,  1,  3,  8, 13, 11,  9],
       [ 0,  3,  1,  5,  4,  6, 11, 10,  8],
       [ 0,  1,  0,  3,  2,  7,  9,  8,  7]])

In [9]:
print('CIS 545 Test Case Padding')

CIS 545 Test Case Padding


In [10]:
# CIS 545 Test Case


In [11]:
print('CIS 545 Test Case Padding')

CIS 545 Test Case Padding


Now, find the indices of the cell in the score matrix `D_arr` obtained above which has the maximum score. Name the row index as `start_row_idx` and column index as `start_col_idx`.

For simplicity, we assume that we are just looking for one local alignment (There could be many local alignments feasible.) So, if there exists multiple cells with the same highest score, pick the cell which is closer to the extreme right most-bottom most cell $D(m,n)$.

For example, suppose there is a 4x4 matrix with row indices as [0,1,2,3] and column indices as [0,1,2,3] in which cell values at (2,3) and (3,2) contain the same largest score, then pick the indices as (3,2).


In [12]:
start_row_idx=0
start_col_idx=0
local_max=0
for i in range(len(D_arr)):
    for j in range(len(D_arr[0])):
        if D_arr[i][j]>=local_max:
            local_max=D_arr[i][j]
            start_row_idx=i
            start_col_idx=j

In [13]:
print(start_row_idx,start_col_idx,local_max)

7 6 13


In [14]:
# CIS 545 Test Case


In [15]:
print('CIS 545 Test Case Padding')

CIS 545 Test Case Padding


In [16]:
# CIS 545 Test Case


In [17]:
print('CIS 545 Test Case Padding')

CIS 545 Test Case Padding


## Step 1.2: Backtracking to Compute Alignments

Up until now, we have simply been computing the score of the sequence alignments, but we have not yet implemented a way to _find_ a local sequence alignment. We will do so by **“backtracking”** through our alignment score table, essentially starting with the cell which has the maximum score of $D$ and working our way back until a cell with score zero is encountered.

Because of how the dynamic programming recursion was set up, at each cell we visit during our backtrack, we have three options for the next move: 

1. Diagonal (`DIAG`): 
Corresponds to the case in which we either substitute or having matching characters (i.e., the following had the maximum value of the three cases):  
`D(i - 1, j - 1) + match_mismatch_cost(seq1_{i-1}, seq2_{j-1})`
2. Horizontal (`LEFT`):
Corresponds to the case in which we insert a gap in the side sequence:  
`D(i - 1, j) - indelCost`
3. Vertical (`UP`): 
Corresponds to the case in which we insert a gap in the top sequence:  
`D(i, j - 1) - indelCost`


### Step 1.2.1 Computing the Directional Alignment Table
Define a function `determine_directions(seq1, seq2, indelCost)` that returns a dictionary, `D`, that corresponds to the table of alignment scores and the directional arrows computed by the Smith–Waterman algorithm. 

The dictionary should have int tuples `(i, j)` as keys and, as the corresponding mappings, a two-element list of the form `[int, list]`, where int refers to the score from 1.1, and the inner list will hold the directional arrows pointing to the cells in which the maximum score was computed from. An element of the dictionary should resemble the following:

`(5, 5): [0, ['UP', 'DIAG']]`

In [18]:
# Import collections (don't delete this!) and other packages neede
from collections import defaultdict

In [19]:
# TODO: Write the function determine_directions(seq1, seq2, indelCost) here.

def determine_directions(seq1, seq2, indelCost):
    D=dict()
    len1 = len(seq1)
    len2 = len(seq2)
    for i in range(len1+1):
        D[(i,0)]=[0,[]]
    for j in range(len2+1):
        D[(0,j)]=[0,[]]
    for i in range(1,len1+1):
        for j in range(1,len2+1):
            D[(i,j)] = [0,[]]
            D[(i,j)][0] = max((D[(i-1,j)][0]-indelCost),(D[(i,j-1)][0]-indelCost),(D[(i-1,j-1)][0]+match_mismatch_cost(seq1[i-1],seq2[j-1])),0)
            if (D[(i-1,j)][0]-indelCost) == D[(i,j)][0]:
                D[(i,j)][1].append('VERT')
            if (D[(i,j-1)][0]-indelCost) == D[(i,j)][0]:
                D[(i,j)][1].append('HORI')
            if (D[(i-1,j-1)][0]+match_mismatch_cost(seq1[i-1],seq2[j-1])) == D[(i,j)][0]:
                D[(i,j)][1].append('DIAG')
#             print(D[(i,j)][0])
    return D                 

In [20]:
# CIS 545 Test Case

indelCost = 2
D = determine_directions(seq1, seq2, indelCost)
D_arr = directions_grid(D, seq1, seq2)
display(D_arr)

assert((D_arr[4] == [[], [], [], ['HORI', 'DIAG'], ['DIAG'], ['VERT'], ['VERT'], ['VERT'], ['DIAG'], ['VERT', 'DIAG']]).all())

,0,1,2,3,4,5,6,7,8
0,[],[],[],[],[],[],[],[],[]
1,[],[],[DIAG],[HORI],[],[],[],[DIAG],[DIAG]
2,[],[],[DIAG],[HORI],[],[],[],[DIAG],[DIAG]
3,[],[DIAG],"[VERT, HORI]",[DIAG],"[HORI, DIAG]",[HORI],[HORI],[VERT],[VERT]
4,[],[DIAG],[HORI],"[VERT, DIAG]",[DIAG],[HORI],[HORI],[HORI],[VERT]
5,[],[VERT],[DIAG],[HORI],[VERT],[DIAG],"[HORI, DIAG]",[DIAG],"[HORI, DIAG]"
6,[],[],[VERT],[DIAG],[VERT],[DIAG],[HORI],"[VERT, HORI]",[DIAG]
7,[],[],[VERT],"[VERT, DIAG]",[VERT],[VERT],[DIAG],[HORI],[HORI]
8,[],[DIAG],[HORI],[DIAG],[DIAG],[VERT],[VERT],[DIAG],"[HORI, DIAG]"
9,[],[VERT],[DIAG],[VERT],"[VERT, DIAG]",[DIAG],[VERT],"[VERT, DIAG]",[DIAG]


In [21]:
print('CIS 545 Test Case Padding')

CIS 545 Test Case Padding


In [22]:
# CIS 545 Test Case


In [23]:
print('CIS 545 Test Case Padding')

CIS 545 Test Case Padding


### Step 1.2.2 Computing the Paths

Define a function `build_paths(D, r, c, directions)` that returns a list of paths from $D(start\_row\_idx, start\_col\_idx)$ to that cell of $D(i, j)$ where $D(i, j) = 0$ is encountered for the first time. This list of paths should be returned as a list of lists of strings (i.e., where strings can take values as 'DIAG' or 'VERT' or 'HORI').

In [24]:
def build_paths(D, r, c, directions):
    directions_list=[[]]
    next_list = [[r,c]]
    result = []
    while True:
        if len(directions_list)!=0:
            direction = directions_list.pop(0)
            next_ = next_list.pop(0)
#             print(next_,D[(next_[0],next_[1])][0])
            if D[(next_[0],next_[1])][0]==0:
                result.extend([direction])
            else:
#                 print(D[(next_[0],next_[1])][1])
                for item in D[(next_[0],next_[1])][1]:
#                     print(item)
                    direction_new = direction.copy()
                    direction_new.append(item)
#                     print(item)
                    if item == 'VERT':
                        next_new = [next_[0]-1,next_[1]]
                    if item == 'HORI':
                        next_new = [next_[0],next_[1]-1]
                    if item == 'DIAG':
                        next_new = [next_[0]-1,next_[1]-1]
#                     print(direction_new,next_new)
                    directions_list.extend([direction_new]) 
                    next_list.extend([next_new])
#                 print(directions_list,next_list)
        else:
#             print(result)
            return result


In [25]:
# CIS 545 Test Case

indelCost = 2
D = determine_directions(seq1, seq2, indelCost)
directions = build_paths(D, start_row_idx, start_col_idx, [])

display(directions)

assert(directions[0] == ['DIAG', 'DIAG', 'VERT', 'DIAG', 'DIAG', 'DIAG'])

[['DIAG', 'DIAG', 'VERT', 'DIAG', 'DIAG', 'DIAG']]

In [26]:
print('CIS 545 Test Case Padding')

CIS 545 Test Case Padding


In [27]:
# CIS 545 Test Case


In [28]:
print('CIS 545 Test Case Padding')

CIS 545 Test Case Padding


### Step 1.2.3 Computing the Alignments

Finally, now that we have the paths from start to finish, we can compute their corresponding alignments.

Define a function `build_alignments(paths, seq1, seq2, r, c)` that returns the alignments specified by the directional arrows.

`paths` is the output of the `build_paths` function, `r` refers to start\_row\_idx and `c` refers to start\_col\_idx.

`build_alignments` should return a **list of two-element lists**, in which the first element of the **inner list** should refer to the alignment of `seq2`, and the second element should refer to the alignment of `seq1`. You should have as many alignment pairs as paths. Indicate a **gap** in the alignment with '-'.

As you develop your code, you can refer to the “Trace Arrows back to origin” section of the [Wikipedia](https://en.wikipedia.org/wiki/Needleman%E2%80%93Wunsch_algorithm#Tracing_arrows_back_to_origin) article as your guide but remember that in Smith–Waterman algorithm, traceback procedure starts at the highest scoring matrix cell and proceeds until a cell with score zero is encountered, yielding the highest scoring local alignment.

In [29]:
# TODO: Write your build_alignments(paths, seq1, seq2, r, c) function here.

def build_alignments(paths, seq1, seq2, r, c):
    result_seq1=[]
    result_seq2=[]
    for item in paths:
        sub_seq1=[]
        sub_seq2=[]
        r_new=r
        c_new=c
#         print(item)
        for i in item:
            if i == 'DIAG':
                sub_seq1.append(seq1[r_new-1])
                sub_seq2.append(seq2[c_new-1])
                r_new=r_new-1
                c_new=c_new-1
            if i == 'VERT':
                sub_seq1.append(seq1[r_new-1])
                sub_seq2.append('-')
                r_new=r_new-1
            if i == 'HORI':
                sub_seq1.append('-')
                sub_seq2.append(seq2[c_new-1])
                c_new=c_new-1
#             print(sub_seq1,sub_seq2)
        sub_seq1.reverse()
        sub_seq2.reverse()
        result_seq1.extend([sub_seq1])
        result_seq2.extend([sub_seq2])
    return [result_seq2,result_seq1]
            

In [30]:
# CIS 545 Test Case

indelCost = 2
D = determine_directions(seq1, seq2, indelCost)

directions = build_paths(D, start_row_idx, start_col_idx, [])
alignments = build_alignments(directions, seq1, seq2, start_row_idx, start_col_idx)
display(alignments)

assert(alignments[0][0] == ['G', 'T', 'T', '-', 'A', 'C'])

[[['G', 'T', 'T', '-', 'A', 'C']], [['G', 'T', 'T', 'G', 'A', 'C']]]

In [31]:
print('CIS 545 Test Case Padding')

CIS 545 Test Case Padding


In [32]:
# CIS 545 Test Case


In [33]:
print('CIS 545 Test Case Padding')

CIS 545 Test Case Padding


### Grading Supplement

Before submitting, please make sure that the entire notebook runs successfully from top to bottom passing each given test cell, else the notebook may fail in autograding. We shall run your entire notebook and then inject the test cases after every section. Do ensure that you use the same function and variable names as mentioned in the question.